# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import numpy as np
from os import listdir

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    # initialisation d'un DataFrame
    df = pd.DataFrame(columns=['year', 'name', 'gender', 'births'])
    
    # chargement par itération des fichiers .txt et concaténation
    for i in listdir('names/'):
        
        # vérification du fichier à charger (prévient de la charge des fichiers cachés)
        if i[0] == 'y':
            d_temp = pd.read_csv('names/' + i, header = None)
            d_temp.columns = ['name', 'gender', 'births']
            d_temp['year'] = int(i[3:7])
            df = pd.concat([df, d_temp])
            df = df.sort_values(by = ['year', 'gender', 'births', 'name'], 
                                ascending = [True, True, False, True]).reset_index(drop = True)
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [56]:
def df_names_fr():
    
    # lecture fichier et modification des colonnes
    df = pd.read_csv("nat2020.csv", sep=";", header=None, skiprows = 1,
                     names=['gender','name','year','births'],
                     dtype={'gender':'str','name':'str','births':'int'})
    
    
    #changement de l'ordre des colonnes
    df = df[['year','name','gender','births']]
    
    # remplacement des valeurs du genre
    df['gender'] = df['gender'].map(str).replace(['1', '2'], ['M', 'F'])
    
    # suppression des prénoms rares, vides ou dont la longueur < 2
    df.loc[df['name'].isna().values ,'name'] = 'NA'
    df=df[(df['name']!="_PRENOMS_RARES") & (df['name'].str.len()>1)]
    
    # suppression des années vides et conversion en int
    df = df[df['year'] != 'XXXX']
    
    # passage des prénoms en maj juste pour la première lettre
    df['name'] = df['name'].str.title()
    
    # tri des données
    df = df.sort_values(by = ['year', 'gender', 'births', 'name'], 
                                    ascending = [True, True, False, True]).reset_index(drop = True)

    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [4]:
# taux de change
def df_taux_change(devises):
    
    #lecture fichier
    dt = pd.read_csv('Webstat_Export_20210929.csv', sep= ';')

    #modification du titre des colonnes
    col = [i[-4:-1] for i in dt.columns[1:]]
    dt.columns = ['date'] + col

    # suppression des premières lignes
    dt.drop(dt.index[0:5],0,inplace=True)

    # conversion de la date et passage en index
    dt['date'] = pd.to_datetime(dt['date'], format = "%d/%m/%Y")
    dt.set_index(['date'], inplace = True)

    # conversion des caractères spéciaux et passage en float
    dt = dt.applymap(str).applymap(lambda x: str(x.replace(',','.')))
    dt = dt.apply(pd.to_numeric, errors='coerce')
    
    #création du DataFrame des ou de la devise concernée
    df = pd.DataFrame(dt, columns=devises)
    
    #suppression des lignes qui ne comportent que des NaN
    ind = df.index[df.isnull().any(axis=1)].dropna()
    df.drop(ind, 0, inplace=True)
    
    return df

### Tests

In [52]:
import unittest

class Lesson3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [6]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [55]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson3Tests) ... ok
test_df_names_us (__main__.Lesson3Tests) ... ok
test_df_taux_change (__main__.Lesson3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 235.689s

OK
